In [4]:
# 📦 Import everything
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt

# 📂 Load datasets
df_jm1 = pd.read_csv('../data/JM1.csv')
df_kc1 = pd.read_csv('../data/KC1.csv')

# Fix target
df_jm1['defects'] = df_jm1['defects'].astype(int)
df_kc1['defects'] = df_kc1['defects'].astype(int)

# ⚙️ Prepare JM1
X_jm1 = df_jm1.drop('defects', axis=1)
y_jm1 = df_jm1['defects']

scaler_jm1 = StandardScaler()
X_scaled_jm1 = scaler_jm1.fit_transform(X_jm1)

X_train_jm1, X_test_jm1, y_train_jm1, y_test_jm1 = train_test_split(
    X_scaled_jm1, y_jm1, test_size=0.2, stratify=y_jm1, random_state=42
)

smote = SMOTE(random_state=42)
X_train_smote_jm1, y_train_smote_jm1 = smote.fit_resample(X_train_jm1, y_train_jm1)

# ⚙️ Prepare KC1
X_kc1 = df_kc1.drop('defects', axis=1)
y_kc1 = df_kc1['defects']

scaler_kc1 = StandardScaler()
X_scaled_kc1 = scaler_kc1.fit_transform(X_kc1)

X_train_kc1, X_test_kc1, y_train_kc1, y_test_kc1 = train_test_split(
    X_scaled_kc1, y_kc1, test_size=0.2, stratify=y_kc1, random_state=42
)

X_train_smote_kc1, y_train_smote_kc1 = smote.fit_resample(X_train_kc1, y_train_kc1)

# 🤖 Train models on JM1
logreg_jm1 = LogisticRegression(max_iter=1000).fit(X_train_smote_jm1, y_train_smote_jm1)
rf_model_jm1 = RandomForestClassifier(random_state=42).fit(X_train_smote_jm1, y_train_smote_jm1)
xgb_model_jm1 = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42).fit(X_train_smote_jm1, y_train_smote_jm1)
ann_model_jm1 = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42).fit(X_train_smote_jm1, y_train_smote_jm1)

# 🤖 Train models on KC1
logreg_kc1 = LogisticRegression(max_iter=1000).fit(X_train_smote_kc1, y_train_smote_kc1)
rf_model_kc1 = RandomForestClassifier(random_state=42).fit(X_train_smote_kc1, y_train_smote_kc1)
xgb_model_kc1 = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42).fit(X_train_smote_kc1, y_train_smote_kc1)
ann_model_kc1 = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42).fit(X_train_smote_kc1, y_train_smote_kc1)

print("✅ All models retrained successfully!")

# 📋 Define Error Analysis Function
def error_analysis(model, X_test, y_test, df_original, dataset_name, model_name):
    y_pred = model.predict(X_test)
    
    feature_names = df_original.drop('defects', axis=1).columns
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_original.drop('defects', axis=1))
    
    _, X_test_scaled, _, y_test_split = train_test_split(
        X_scaled, df_original['defects'], test_size=0.2, stratify=df_original['defects'], random_state=42
    )
    
    df_compare = pd.DataFrame(X_test_scaled, columns=feature_names)
    df_compare['True Label'] = y_test_split.values
    df_compare['Predicted Label'] = y_pred

    df_errors = df_compare[df_compare['True Label'] != df_compare['Predicted Label']]
    
    small_modules_errors = df_errors[df_errors['loc'] < 20]
    complex_modules_errors = df_errors[df_errors['v(g)'] > 10]
    
    print(f"\n🧪 Error Analysis - {model_name} on {dataset_name}")
    print(f"Total misclassified samples: {len(df_errors)}")
    print(f"Misclassified small modules (LOC < 20): {len(small_modules_errors)}")
    print(f"Misclassified complex modules (v(g) > 10): {len(complex_modules_errors)}")

# 📈 Run Error Analysis for JM1
error_analysis(logreg_jm1, X_test_jm1, y_test_jm1, df_jm1, "JM1", "Logistic Regression")
error_analysis(rf_model_jm1, X_test_jm1, y_test_jm1, df_jm1, "JM1", "Random Forest")
error_analysis(xgb_model_jm1, X_test_jm1, y_test_jm1, df_jm1, "JM1", "XGBoost")
error_analysis(ann_model_jm1, X_test_jm1, y_test_jm1, df_jm1, "JM1", "ANN (MLPClassifier)")

# 📈 Run Error Analysis for KC1
error_analysis(logreg_kc1, X_test_kc1, y_test_kc1, df_kc1, "KC1", "Logistic Regression")
error_analysis(rf_model_kc1, X_test_kc1, y_test_kc1, df_kc1, "KC1", "Random Forest")
error_analysis(xgb_model_kc1, X_test_kc1, y_test_kc1, df_kc1, "KC1", "XGBoost")
error_analysis(ann_model_kc1, X_test_kc1, y_test_kc1, df_kc1, "KC1", "ANN (MLPClassifier)")


C:\Users\josef\AppData\Roaming\Python\Python313\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] Det går inte att hitta filen
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\josef\AppData\Roaming\Python\Python313\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "c:\Program Files\Python313\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Program Files\Python313\Lib\subprocess.py", line 1036, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
    ~~~~~~~~

✅ All models retrained successfully!

🧪 Error Analysis - Logistic Regression on JM1
Total misclassified samples: 762
Misclassified small modules (LOC < 20): 762
Misclassified complex modules (v(g) > 10): 0

🧪 Error Analysis - Random Forest on JM1
Total misclassified samples: 431
Misclassified small modules (LOC < 20): 431
Misclassified complex modules (v(g) > 10): 1

🧪 Error Analysis - XGBoost on JM1
Total misclassified samples: 438
Misclassified small modules (LOC < 20): 438
Misclassified complex modules (v(g) > 10): 0

🧪 Error Analysis - ANN (MLPClassifier) on JM1
Total misclassified samples: 625
Misclassified small modules (LOC < 20): 625
Misclassified complex modules (v(g) > 10): 1

🧪 Error Analysis - Logistic Regression on KC1
Total misclassified samples: 114
Misclassified small modules (LOC < 20): 114
Misclassified complex modules (v(g) > 10): 0

🧪 Error Analysis - Random Forest on KC1
Total misclassified samples: 64
Misclassified small modules (LOC < 20): 64
Misclassified comple